In this project, we will create a MySQL database for video game sales data cleaned in the previous notebook. This data includes several attributes about each game, such as its platform, year of release, genre, publisher, and others, as well as sales figures from different regions, critic scores, and user scores. This guide will also detail how to create the database, the tables, and how to insert the data.

Let's get started.


Importing the Required Libraries

First, we will import the necessary libraries. This includes mysql.connector to connect to MySQL, pandas to handle our data, numpy to deal with numerical operations, and datetime to handle date-time data.

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import datetime

Defining Auxiliary Functions

We will define some helper functions to handle our data properly.

The first function check_string_length is used to check if a given string's length exceeds a set limit. This helps to ensure data consistency and avoid any errors while inserting data into the MySQL database.

The second function convert_datatype helps in converting a value to a specific data type. This is crucial to match the data types of the columns in our MySQL tables.

In [ ]:
# Function to check if string length exceeds a given limit
def check_string_length(string, limit):
    if pd.isnull(string):
        return True
    return len(str(string)) <= limit

# Function to convert to appropriate data type
def convert_datatype(val, dtype):
    if pd.isnull(val):
        return None
    return dtype(val)

Creating a MySQL Connection


We establish a connection to our MySQL server and  we can create our VideoGamesSales database and the necessary tables. Here, we will create four tables: VideoGame, Sales, CriticReview, and UserReview.

In [ ]:
cnx = mysql.connector.connect(
    host='****',
    user='****',
    password='****',
)

# Create a cursor object
cursor = cnx.cursor()

# Create Database
cursor.execute("CREATE DATABASE IF NOT EXISTS VideoGamesSales")

# Use new Database
cursor.execute("USE VideoGamesSales")

# Create Tables
cursor.execute("""
    CREATE TABLE IF NOT EXISTS VideoGame (
        ID INT PRIMARY KEY AUTO_INCREMENT,
        Name VARCHAR(100),
        Platform VARCHAR(50),
        Year_of_Release INT,
        Genre VARCHAR(50),
        Publisher VARCHAR(100),
        Developer VARCHAR(100),
        Rating VARCHAR(10)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Sales (
        ID INT PRIMARY KEY AUTO_INCREMENT,
        VideoGameID INT,
        NA_Sales DECIMAL(18, 2),
        EU_Sales DECIMAL(18, 2),
        JP_Sales DECIMAL(18, 2),
        Other_Sales DECIMAL(18, 2),
        Global_Sales DECIMAL(18, 2),
        FOREIGN KEY (VideoGameID) REFERENCES VideoGame(ID)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS CriticReview (
        ID INT PRIMARY KEY AUTO_INCREMENT,
        VideoGameID INT,
        Critic_score DECIMAL(18, 2),
        Critic_count INT,
        FOREIGN KEY (VideoGameID) REFERENCES VideoGame(ID)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS UserReview (
        ID INT PRIMARY KEY AUTO_INCREMENT,
        VideoGameID INT,
        User_score DECIMAL(18, 2),
        User_count INT,
        FOREIGN KEY (VideoGameID) REFERENCES VideoGame(ID)
    )
""")


These SQL statements will create the database and tables if they don't already exist. Notice how each table is created with an ID field as the primary key, and we've established foreign key relationships for the Sales, CriticReview, and UserReview tables to the VideoGame table. This design will allow us to maintain data integrity and enforce relationships between the tables.

Loading the Data and Populating the Tables

In this section, we will populate our MySQL database with the video game sales data. The code provided performs the necessary steps to insert the data into the respective tables in the database. Let's go through the process step by step.

The first step is to load the cleaned and imputed data from the CSV file. We use the `pd.read_csv()` function to read the data into a DataFrame.


Next, we iterate over each row in the DataFrame and insert the data into the appropriate tables in the MySQL database. During this process, the code performs data validation and converts the values to the appropriate data types.

The code includes error handling to log and handle any errors that may occur during the iteration and insertion process. MySQL errors, data validation errors, and general exceptions are handled separately to provide appropriate feedback and continue the process.

In [ ]:
# Load the data from CSV
data = pd.read_csv("imputed_data.csv")

# Iterate over each row in the DataFrame
for index, row in data.iterrows():
    try:
        # Check data types and lengths
        assert check_string_length(row['Name'], 255), "Invalid 'Name'"
        assert check_string_length(row['Platform'], 50), "Invalid 'Platform'"
        assert isinstance(row['Year_of_Release'], (int, float, type(None))), "Invalid 'Year_of_Release'"
        assert check_string_length(row['Genre'], 100), "Invalid 'Genre'"
        assert check_string_length(row['Publisher'], 255), "Invalid 'Publisher'"
        assert check_string_length(row['Developer'], 255), "Invalid 'Developer'"
        assert check_string_length(row['Rating'], 10), "Invalid 'Rating'"

        # Convert to appropriate data type
        name = convert_datatype(row['Name'], str)
        platform = convert_datatype(row['Platform'], str)
        if not pd.isnull(row['Year_of_Release']):
            year_of_release = datetime.datetime(int(row['Year_of_Release']), 1, 1) # defaulting to January 1 of the release year
        genre = convert_datatype(row['Genre'], str)
        publisher = convert_datatype(row['Publisher'], str)
        developer = convert_datatype(row['Developer'], str)
        rating = convert_datatype(row['Rating'], str)

        # Construct the SQL query
        query = """INSERT INTO VideoGame (Name, Platform, Year_of_Release, Genre, Publisher, Developer, Rating)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        values = (name, platform, year_of_release, genre, publisher, developer, rating)

        # Execute the query
        cursor.execute(query, values)

        # Get the newly inserted videogame id
        videogame_id = cursor.lastrowid

        na_sales = convert_datatype(row['NA_Sales'], float)
        eu_sales = convert_datatype(row['EU_Sales'], float)
        jp_sales = convert_datatype(row['JP_Sales'], float)
        other_sales = convert_datatype(row['Other_Sales'], float)
        global_sales = convert_datatype(row['Global_Sales'], float)

        query = """
            INSERT INTO Sales (
                VideoGameID, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales
            ) VALUES (
                %s, %s, %s, %s, %s, %s
            )
        """
        values = (videogame_id, na_sales, eu_sales, jp_sales, other_sales, global_sales)

        cursor.execute(query, values)

        critic_score = convert_datatype(row['Critic_Score'], float)
        critic_count = convert_datatype(row['Critic_Count'], int)

        query = """
            INSERT INTO CriticReview (
                VideoGameID, Critic_score, Critic_count
            ) VALUES (
                %s, %s, %s
            )
        """
        values = (videogame_id, critic_score, critic_count)

        cursor.execute(query, values)

        user_score = convert_datatype(row['User_Score'], float)
        user_count = convert_datatype(row['User_Count'], int)

        query = """
            INSERT INTO UserReview (
                VideoGameID, User_score, User_count
            ) VALUES (
                %s, %s, %s
            )
        """
        values = (videogame_id, user_score, user_count)

        cursor.execute(query, values)

        # Commit the transaction after each record to ensure data is saved to the database
        cnx.commit()

    except AssertionError as err:
        # Log the data validation error and continue to next row
        print(f"Data validation error at row {index}: {err}")
        continue
    except mysql.connector.Error as err:
        # Log the MySQL error and continue to next row
        print(f"MySQL error at row {index}: {err}")
        continue
    except Exception as err:
        # Log any other error and continue to next row
        print(f"Other error at row {index}: {err}")
        continue

# Close the cursor and connection
cursor.close()
cnx.close()